Note: Pytorch模型训练实用教程（余霆嵩）
----
<https://github.com/tensor-yu/PyTorch_Tutorial>
Chapter4: 监控模型——可视化
---

**4.4 梯度及权值分布可视化**   
1. 在网络训练过程中，我们常常会遇到梯度消失、梯度爆炸等问题，我们可以通过记录每个epoch的梯度的值来检测梯度的情况；
2. 还可以记录权值，分析权值更新的方向是否符合规律。

In [39]:
import torch
from torch.utils.data import DataLoader
import torchvision.transforms as transforms
import numpy as np
import os
from torch.autograd import Variable
import torch.nn as nn
import torch.optim as optim
import sys
import os
sys.path.append("..")
from utils import MyDataset, validate, show_confMat, Net
from tensorboardX import SummaryWriter
from datetime import datetime

In [49]:
train_txt_path = os.path.join("Data","train.txt")
valid_txt_path = os.path.join("Data", "valid.txt")
classes_name = ['plane','car','bird','cat','deer','dog','frog','horse','ship','truck']
train_bs = 16
valid_bs = 16
lr_init = 0.001
max_epoch = 3
# log
log_dir = os.path.join("Result","hist_grad_weight")
writer = SummaryWriter(log_dir = log_dir)

**Step 1/4: 加载数据**

In [50]:
# 数据预处理
norMean = [0.4948052, 0.48568845, 0.44682974]
normStd = [0.24580306, 0.24236229, 0.2603115]
normTransform = transforms.Normalize(normMean, normStd)
trainTransform = transforms.Compose([
    transforms.Resize(32),
    transforms.RandomCrop(32, padding=4),
    transforms.ToTensor(),
    normTransform
])
validTransform = transforms.Compose([
    transforms.ToTensor(),
    normTransform
])

# 构建Dataset实例
train_data = MyDataset(txt_path=train_txt_path, transform=trainTransform)
valid_data = MyDataset(txt_path=valid_txt_path, transform=validTransform)

#构建DataLoder
train_loader = DataLoader(dataset=train_data, batch_size=train_bs, shuffle=True)
valid_loader = DataLoader(dataset=valid_data, batch_size=valid_bs)

**Step 2/4: 网络初始化**

In [51]:
net = Net() #创建一个网络
net.initialize_weights() #初始化权值

**Step 3/4: 定义损失函数和优化器**

In [44]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=lr_init, momentum=0.9, dampening=0.1)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=50, gamma=0.1) # 设置学习率下降策略

**Step 4/4: 训练**

In [ ]:
for epoch in range(max_epoch):
    loss_sigma = 0.0 #记录一个epoch的loss之和
    correct = 0.0
    total = 0.0
    
    
    for i, data in enumerate(train_loader):
        # 获取图片和标签
        inputs, labels = data
        inputs, labels = Variable(inputs), Variable(labels)
        # forward, backward, update weights
        optimizer.zero_grad()
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        
        # 统计预测信息
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct =+(predicted == labels).squeeze().sum().numpy()
        loss_sigma +=loss.item()
        
        # 每10个iteration打印一次训练信息，loss为10个iteration的平均
        if i% 10 == 9:
            loss_avg = loss_sigma/10
            loss_sigma = 0.0
            print("Training:Epoch[{:0>3}/{:0>3}] Iteration[{:0>3}/{:0>3}] Loss:{:.4f} Acc:{:.2%}".format(
            epoch+1, max_epoch, i+1, len(train_loader), loss_avg, correct/total))
    scheduler.step() #更新学习率
    # 每个epoch，记录梯度，权值
    for name, layer in net.named_parameters():
        writer.add_histogram(name+ '_grad', layer.grad.cpu().data.numpy(), epoch)
        writer.add_histogram(name+ '_data', layer.cpu().data.numpy(), epoch)
print('Finished Training')
        

**1: 权值weights的监控**  
(1) 卷积层的权值随着训练不断的“扩散”，扩大，一开始是个比较标准的高斯分布，并且最大值不会超过0.3；   
(2) 到了后期，权值会发散到0.6+, 这个问题也是需要关注的，若权值太大容易导致过拟合。   
(3) 因为模型的输出值会被该特征所主导，从而引起过拟合现象，这个可以通过权值衰减(weight_decay)来缓解。    
**2：偏置bias的监控**   
(1) 通常会监控输出层的bias的大小，若有特别大的，或者特别小的bias, 那么某一类别的召回率可能会很低。    
(2) 可以通过观察输出层的bias来诊断是否在这一环节出问题。   
**3: 梯度的监控**   
(1) 倘若前面几层的梯度非常小，那么就是梯度流通不畅导致的，可以考虑残差结构或者辅助损失层等trick来解决梯度消失。

**文末思考**   
(1) 为梯度小的值设置更大的学习率。   
(2) 权值大的层，可以考虑设置更大的weight_decay,是否能有效降低该层的权值大小呢？   